# GPT4oを用いた画像処理について試す

## Install

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


## Environment

In [1]:
import os

AZURE_OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.environ.get('OPENAI_API_BASE')
DEPLOYMENT_NAME = os.environ.get('OPENAI_DEPLOYMENT_NAME_GPT4o')
API_VERSION = os.environ.get('OPENAI_API_VERSION')

In [4]:
AZURE_OPENAI_ENDPOINT

'https://trial-openai.openai.azure.com/'

## Usage

参考：[公式サンプルコード](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/gpt-v-quickstart?tabs=image%2Ccommand-line&pivots=programming-language-python#create-a-new-python-application)

In [2]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,  
    api_version=API_VERSION,
    base_url=f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}"
)

response = client.chat.completions.create(
    model=DEPLOYMENT_NAME,
    messages=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Describe this picture:" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "https://learn.microsoft.com/ja-jp/azure/ai-services/openai/media/quickstarts/endpoint.png#lightbox"
                }
            }
        ] } 
    ],
    # max_tokens=2000 　GPT4oはデフォルト4096の為指定不要
)

print(response)

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}